# Assignment 1: Data Acquisition with APIs and Scraping
## Kevon Cambridge
### Artist picked:
1. Dermot Kennedy
2. Damian Marley

##### Importing required libraries

In [1]:
#Twitter Section
import tweepy
import os
import datetime
import re
from pprint import pprint
import pandas as pd

#Lyrics Scrape Section
import requests
import time
import random
from bs4 import BeautifulSoup
from collections import defaultdict, Counter

##### API Key Requirements

In [2]:
from kevon_twitter_api_keys import api_key, api_key_secret, bearer_token

In [3]:
client = tweepy.Client(bearer_token, wait_on_rate_limit = True)

##### Testing API

In [4]:
handle = "37chandler"
user_obj = client.get_user(username=handle)

followers = client.get_users_followers(
    # Learn about user fields here: 
    # https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/user
    user_obj.data.id, user_fields=["created_at","description","location",
                                   "public_metrics"]
)

In [5]:
handle

'37chandler'

##### Printing out names, locations, following counts and followers count.

In [6]:
num_to_print = 20

for idx, user in enumerate(followers.data) :
    following_count = user.public_metrics['following_count']
    followers_count = user.public_metrics['followers_count']
    
    print(f"{user.name} lists '{user.location}' as their location.")
    print(f" Following: {following_count}, Followers: {followers_count}.")
    print()
    
    if idx >= (num_to_print - 1) :
        break

Dave Renn lists 'None' as their location.
 Following: 42, Followers: 10.

Lionel lists 'None' as their location.
 Following: 202, Followers: 204.

Megan Randall lists 'None' as their location.
 Following: 141, Followers: 100.

Jacob Salzman lists 'None' as their location.
 Following: 562, Followers: 134.

twiter not fun lists 'None' as their location.
 Following: 221, Followers: 21.

Hariettwilsonincarnate lists 'None' as their location.
 Following: 218, Followers: 60.

Christian Tinsley lists 'None' as their location.
 Following: 2, Followers: 0.

Steve lists 'I'm over here.' as their location.
 Following: 1591, Followers: 33.

John O'Connor 🇺🇦 lists 'None' as their location.
 Following: 8, Followers: 1.

CodeGrade lists 'Amsterdam' as their location.
 Following: 2820, Followers: 424.

Cleverhood lists 'Providence, RI' as their location.
 Following: 2795, Followers: 3563.

Regina 🚶‍♀️🚲🌳 lists 'Minneapolis' as their location.
 Following: 2802, Followers: 3336.

Eric Hallstrom lists 'Mi

##### Finding person with the most followers who follows this handle

In [7]:
max_followers = 0

for idx, user in enumerate(followers.data) :
    followers_count = user.public_metrics['followers_count']
    
    if followers_count > max_followers :
        max_followers = followers_count
        max_follower_user = user

        
print(max_follower_user)
print(max_follower_user.public_metrics)

WedgeLIVE
{'followers_count': 14180, 'following_count': 2221, 'tweet_count': 56103, 'listed_count': 218}


##### printing out user_fields

In [8]:
response = client.get_user(id=user_obj.data.id,
                          user_fields=["created_at","description","location",
                                       "entities","name","pinned_tweet_id","profile_image_url",
                                       "verified","public_metrics"])

In [9]:
for field, value in response.data.items() :
    print(f"for {field} we have {value}")

for location we have MN
for username we have 37chandler
for name we have John Chandler
for id we have 33029025
for public_metrics we have {'followers_count': 193, 'following_count': 589, 'tweet_count': 997, 'listed_count': 3}
for description we have He/Him. Data scientist, urban cyclist, educator, erstwhile frisbee player. 

¯\_(ツ)_/¯
for created_at we have 2009-04-18 22:08:22+00:00
for profile_image_url we have https://pbs.twimg.com/profile_images/2680483898/b30ae76f909352dbae5e371fb1c27454_normal.png
for verified we have False


###### Q: How many fields are being returned in the response Object?
A: We have 8 fields being returned in the response object

###### Q: Are any of the fields within the user object non-scalar? 
A: Yes, the created_at is a non-scalar object. It is a datetime datatype.

###### Q: How many friends, followers, and tweets does this user have?
A: This person have 193 followers and friends and 994 tweets.

##### Looking at a few tweets from this user

In [10]:
response = client.get_users_tweets(user_obj.data.id)

# By default, only the ID and text fields of each Tweet will be returned
for idx, tweet in enumerate(response.data) :
    print(tweet.id)
    print(tweet.text)
    print()
    
    if idx > 10 :
        break

1569760631548690437
RT @dtmooreeditor: So there's a particular quirk of English grammar that I've always found quite endearing: the exocentric verb-noun compou…

1569155273742327811
As a Minneapolis person, I knew we had Toronto beat, but I didn't realize Portland had us beat: https://t.co/xrx5mOFcWK.

But @nytimes, c'mon! https://t.co/M9mBWhdgsj

1568982292923826176
RT @wonderofscience: Amazing lenticular cloud over Mount Fuji

Credit: Iurie Belegurschi
https://t.co/0mUxl28H9U

1568242374085869570
RT @depthsofwiki: lots of memes about speedy wikipedia editors — quick thread about what went down on wikipedia in the minutes after her de…

1568074978754703361
@DrLaurenWilson @leighradwood @MaritsaGeorgiou @Walgreens I could not possibly agree more with this sentiment. Compared to almost any other primary care I've received, they are great.

1567530169686196224
@DrLaurenWilson @MaritsaGeorgiou @Walgreens For those who have access to Curry Health Center on campus, you can get a bivalent bo

### Pulling Follower Information

##### Rate Limiting

###### This block of code was really helpful for my situation

In [11]:
handle_followers = []
pulls = 0
max_pulls = 100
next_token = None

while True :

    followers = client.get_users_followers(
        user_obj.data.id, 
        max_results=1000, # when you do this for real, set this to 1000!
        pagination_token = next_token,
        user_fields=["created_at","description","location",
                     "entities","name","pinned_tweet_id","profile_image_url",
                     "verified","public_metrics"]
    )
    pulls += 1
    
    for follower in followers.data : 
        follower_row = (follower.id,follower.name,follower.created_at,follower.description)
        handle_followers.append(follower_row)
    
    if 'next_token' in followers.meta and pulls < max_pulls :
        next_token = followers.meta['next_token']
    else : 
        break

#### Pulling Twitter Data for Dermot Kennedy & Damian Marley

In [12]:
artists = dict()
handles = ['DermotKennedy','damianmarley']
for handle in handles: 
    user_obj = client.get_user(username=handle,user_fields=["public_metrics"])
    artists[handle] = (user_obj.data.id, 
                       handle,
                       user_obj.data.public_metrics['followers_count'])
    

for artist, data in artists.items() : 
    print(f"It would take {data[2]/(1000*15*4):.2f} hours to pull all {data[2]} followers for {artist}. ")

It would take 1.80 hours to pull all 107940 followers for DermotKennedy. 
It would take 9.07 hours to pull all 544394 followers for damianmarley. 


In [13]:
artists

{'DermotKennedy': (147529097, 'DermotKennedy', 107940),
 'damianmarley': (245871948, 'damianmarley', 544394)}

#### Making folder to write data

In [14]:
dirname = "twitter"

if not os.path.isdir(dirname) : 
    #shutil.rmtree("twitter/")
    os.mkdir("twitter")

#Checking to see if the folder exist
isdir = os.path.isdir(dirname)
print(isdir)

True


#### Writing artist data to twitter folder

In [19]:
#Limiting number of followers to pull. We are limiting to 200,000 followers
num_followers_to_pull = 200*1000

##### Pulling data and inserting in text file

In [20]:
# Modify the below code stub to pull the follower IDs and write them to a file. 
handles = ['damianmarley','DermotKennedy']
whitespace_pattern = re.compile(r"\s+")

user_data = dict() 
followers_data = dict()


# Grabs the time when we start making requests to the API
start_time = datetime.datetime.now()
for handle in handles :
    pulls = 0
    user_data[handle] = []
    followers_data[handle] = [] # will be a simple list of IDs
    # Create the output file names 

    followers_output_file = "twitter/" + handle + "_followers.txt"
    users_output_file = "twitter/" + handle + "_followers_data.txt"
    user_obj = client.get_user(username = handle)
    print(f'pulling followers for {handle}.')
    
    
        
    for followers in tweepy.Paginator(client.get_users_followers, user_obj.data.id, 
                                      user_fields=['id','username','name','description','location','public_metrics']
                                      ,max_results = 1000, limit = 100):
        
        pulls +=1
        print(f'pulled {pulls+1} for {handle}...')
        
        for user in followers.data:
            users = (str(user.username),
                     str(user.name),
                     str(user.location),
                     re.sub(r"\s+"," ", user.description).strip(),
                     str(user.public_metrics['followers_count']),
                     str(user.public_metrics['following_count']))
            user_data[handle].append(users)
        
            followersid = (str(user.id))
            followers_data[handle].append(followersid)
                
        #for follower in followers.data:
        #fols = (followers.id)
        #followers_data[handle].append(fols)
            
            
        # If you've pulled num_followers_to_pull, feel free to break out paged twitter API response
        if len(followers_data[handle]) >= num_followers_to_pull:
            break
            
        time.sleep(60)
        # Writing the IDs to the output file in the `twitter` folder
        with open(users_output_file, 'w', encoding='utf-8') as f1:
            headers = '\t'.join(['username','name','location','description','followers_count','following_count'])
            f1.write(headers+"\n")
            for user in followers_data:
                row = handle + "\t" + "\t".join(str(i) for i in user_data[handle])
                f1.write(row+"\n")
        with open(followers_output_file, 'w', encoding='utf-8') as f2:
            headers = '\t'.join(['id'])
            f2.write(headers+"\n")
            for follower in followers_data:
                row = handle + "\t" + "\t".join(str(i) for i in followers_data[handle])
                f2.write(row+"\n")
    print(f'finished pulling followers for {handle}.')
       
# Let's see how long it took to grab all follower IDs
end_time = datetime.datetime.now()
print(end_time - start_time)

pulling followers for damianmarley.
pulled 141 for damianmarley...
pulled 142 for damianmarley...
pulled 143 for damianmarley...
pulled 144 for damianmarley...
pulled 145 for damianmarley...
pulled 146 for damianmarley...
pulled 147 for damianmarley...
pulled 148 for damianmarley...
pulled 149 for damianmarley...
pulled 150 for damianmarley...
pulled 151 for damianmarley...
pulled 152 for damianmarley...
pulled 153 for damianmarley...


Rate limit exceeded. Sleeping for 13 seconds.


pulled 154 for damianmarley...
pulled 155 for damianmarley...
pulled 156 for damianmarley...
pulled 157 for damianmarley...
pulled 158 for damianmarley...
pulled 159 for damianmarley...
pulled 160 for damianmarley...
pulled 161 for damianmarley...
pulled 162 for damianmarley...
pulled 163 for damianmarley...
pulled 164 for damianmarley...
pulled 165 for damianmarley...
pulled 166 for damianmarley...
pulled 167 for damianmarley...
pulled 168 for damianmarley...
pulled 169 for damianmarley...
pulled 170 for damianmarley...
pulled 171 for damianmarley...
pulled 172 for damianmarley...
pulled 173 for damianmarley...
pulled 174 for damianmarley...
pulled 175 for damianmarley...
pulled 176 for damianmarley...
pulled 177 for damianmarley...
pulled 178 for damianmarley...
pulled 179 for damianmarley...
pulled 180 for damianmarley...
pulled 181 for damianmarley...
pulled 182 for damianmarley...
pulled 183 for damianmarley...
pulled 184 for damianmarley...
pulled 185 for damianmarley...
pulled 1

In [21]:
tricky_description = """
    Home by Warsan Shire
    
    no one leaves home unless
    home is the mouth of a shark.
    you only run for the border
    when you see the whole city
    running as well.

"""
# This won't work in a tab-delimited text file.

clean_description = re.sub(r"\s+"," ",tricky_description).strip()
clean_description

'Home by Warsan Shire no one leaves home unless home is the mouth of a shark. you only run for the border when you see the whole city running as well.'

## Lyrics Scrape

In [22]:
artists = {'damianmarley':"https://www.azlyrics.com/d/damianmarley.html",
           'DermotKennedy':"https://www.azlyrics.com/d/dermotkennedy.html"} 
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

## A Note on Rate Limiting
The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to a Tom Jones song.)

Whenever you call requests.get to retrieve a page, put a time.sleep(5 + 10*random.random()) on the next line. This will help you not to get blocked. If you do get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again.

#### Finding links to songs lyrics

In [98]:

# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items() :
    # request the page and sleep
    r = requests.get(artist_page)
    time.sleep(5 + 10*random.random())

    # now extract the links to lyrics pages from this page
    soup = BeautifulSoup(r.text, 'html.parser')
    #for i in soup.find_all('div', {'class': 'listalbum-item'}):
        #print(i.find('a')['href'])
    links_list = soup.find_all('div', {'class': 'listalbum-item'})
    for div in links_list:
        lyrics_pages[artist].append(div.find('a')['href'])
    #for link in links_list:
        #lyrics_pages[artist].append(link.get('href'))
        #print(link.get('href'))
    # store the links `lyrics_pages` where the key is the artist and the
    # value is a list of links. 
    

In [99]:
lyrics_pages.items()

dict_items([('damianmarley', ['/lyrics/damianmarley/trouble.html', '/lyrics/damianmarley/loveandinity.html', '/lyrics/damianmarley/10000chariots.html', '/lyrics/damianmarley/oldwarchant.html', '/lyrics/damianmarley/partytime.html', '/lyrics/damianmarley/kingston12.html', '/lyrics/damianmarley/keepongrooving.html', '/lyrics/damianmarley/searchingsomuchbubble.html', '/lyrics/damianmarley/onemorecupofcoffee.html', '/lyrics/damianmarley/julie.html', '/lyrics/damianmarley/menamejrgong.html', '/lyrics/damianmarley/mrmarley.html', '/lyrics/damianmarley/educatedfools.html', '/lyrics/damianmarley/morejustice.html', '/lyrics/damianmarley/itwaswritten.html', '/lyrics/damianmarley/catchafire.html', '/lyrics/damianmarley/stillsearchin.html', '/lyrics/damianmarley/sheneedsmylove.html', '/lyrics/damianmarley/miblenda.html', '/lyrics/damianmarley/whereisthelove.html', '/lyrics/damianmarley/harderinterlude.html', '/lyrics/damianmarley/borntobewild.html', '/lyrics/damianmarley/givedemsomeway.html', '/ly

In [100]:
for artist, lp in lyrics_pages.items():
    assert(len(set(lp)) > 20) 

In [101]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For damianmarley we have 89.
The full pull will take for this artist will take 0.25 hours.
For DermotKennedy we have 59.
The full pull will take for this artist will take 0.16 hours.


#### Pulling Lyrics
Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part.

    1. Create an empty folder in our repo called "lyrics".
    2. Iterate over the artists in lyrics_pages.
    3. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have lyrics/cher/ in your repo.
    4. Iterate over the pages.
    5. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
    6. Use the function below, generate_filename_from_url, to create a filename based on the lyrics page, then write the lyrics to a text file with that name.

In [102]:
def generate_filename_from_link(link) :
    
    if not link :
        return None
    
    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    name = link.replace(".html","")

    name = name.replace("/lyrics/","")
    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return(name)

In [109]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.
dirname = 'lyrics'
if not os.path.isdir(dirname): 
    #shutil.rmtree("lyrics/")
     os.mkdir("lyrics")
    
isdir = os.path.isdir(dirname)
print(isdir)

True


In [116]:
url_stub = "https://www.azlyrics.com" 
start = time.time()

total_pages = 0 

for artist in lyrics_pages :

    # Use this space to carry out the following steps: 
    # 1. Build a subfolder for the artist
    artist_folder = os.path.join('lyrics', artist)
    
    if not os.path.isdir(artist_folder):
        os.mkdir(artist_folder)
    # 2. Iterate over the lyrics pages
    for song, link in enumerate(lyrics_pages[artist]):
        
        song_file = generate_filename_from_link(link)
        print('The name of the file is: ', song_file)
        leek = os.path.join('lyrics' ,artist, song_file)
        
    # 3. Request the lyrics page. 
        lyrics_link = url_stub + link.replace('..','')
        print('\t'+lyrics_link)
        req = requests.get(lyrics_link, timeout = 5)
        soup = BeautifulSoup(req.content, 'html.parser')
        req.close()
        
        # Don't forget to add a line like `time.sleep(5 + 10*random.random())`
        # to sleep after making the request
        time.sleep(5 + 10*random.random())
        
    # 4. Extract the title and lyrics from the page.
        all_title = soup.findAll("title")
        song_title = soup.title.text.split("-")[1].split("|")[0].replace("Lyrics","").strip()
        start = soup.text.find(f'"{song_title}"')
        end = soup.text.find('Submit Corrections')
        
    # 5. Write out the title, two returns ('\n'), and the lyrics. Use `generate_filename_from_url`
    #    to generate the filename. 
        #song_file = generate_filename_from_link(link)
        #print('The name of the file is: ', song_file)
        #leek = os.path.join('lyrics' ,artist, song_file)
        with open(leek, 'w') as file:
            file.write(song_title + '\n\n')
            for line in soup.text[start:end].split('\n')[11:]:
                file.write(line + '\n')
        file.close()
    # Remember to pull at least 20 songs per artist. It may be fun to pull all the songs for the artist
        if song >= 20:
                break

The name of the file is:  damianmarley_trouble.txt
	https://www.azlyrics.com/lyrics/damianmarley/trouble.html
The name of the file is:  damianmarley_loveandinity.txt
	https://www.azlyrics.com/lyrics/damianmarley/loveandinity.html
The name of the file is:  damianmarley_10000chariots.txt
	https://www.azlyrics.com/lyrics/damianmarley/10000chariots.html
The name of the file is:  damianmarley_oldwarchant.txt
	https://www.azlyrics.com/lyrics/damianmarley/oldwarchant.html
The name of the file is:  damianmarley_partytime.txt
	https://www.azlyrics.com/lyrics/damianmarley/partytime.html
The name of the file is:  damianmarley_kingston12.txt
	https://www.azlyrics.com/lyrics/damianmarley/kingston12.html
The name of the file is:  damianmarley_keepongrooving.txt
	https://www.azlyrics.com/lyrics/damianmarley/keepongrooving.html
The name of the file is:  damianmarley_searchingsomuchbubble.txt
	https://www.azlyrics.com/lyrics/damianmarley/searchingsomuchbubble.html
The name of the file is:  damianmarley

In [117]:

print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

Total run time was 461982.02 hours.


### Checking Twitter Data
The output from your Twitter API pull should be two files per artist, stored in files with formats like cher_followers.txt (a list of all follower IDs you pulled) and cher_followers_data.txt. These files should be in a folder named twitter within the repository directory. This code summarizes the information at a high level to help the instructor evaluate your work.

In [118]:
twitter_files = os.listdir("twitter")
twitter_files = [f for f in twitter_files if f != ".DS_Store"]
artist_handles = list(set([name.split("_")[0] for name in twitter_files]))

print(f"We see two artist handles: {artist_handles[0]} and {artist_handles[1]}.")

We see two artist handles: damianmarley and DermotKennedy.


In [131]:
for artist in artist_handles :
    follower_file = artist + "_followers.txt"
    follower_data_file = artist + "_followers_data.txt"
    
    ids = open("twitter/" + follower_file,'r').readlines()
    
    print(f"We see {len(ids)-1} in your follower file for {artist}, assuming a header row.")
    
    with open("twitter/" + follower_data_file,'r', encoding='utf-8') as infile :
        
        # check the headers
        headers = infile.readline().split("\t")
        
        print(f"In the follower data file ({follower_data_file}) for {artist}, we have these columns:")
        print(" : ".join(headers))
        
        description_words = []
        locations = set()
        
        
        for idx, line in enumerate(infile.readlines()) :
            line = line.strip("\n").split("\t")
            
            try : 
                locations.add(line[3])            
                description_words.extend(words(line[6]))
            except :
                pass
    
        

        print(f"We have {idx+1} data rows for {artist} in the follower data file.")

        print(f"For {artist} we have {len(locations)} unique locations.")

        print(f"For {artist} we have {len(description_words)} words in the descriptions.")
        print("Here are the five most common words:")
        print(Counter(description_words).most_common(5))

        
        print("")
        print("-"*40)
        print("")

We see 1 in your follower file for damianmarley, assuming a header row.
In the follower data file (damianmarley_followers_data.txt) for damianmarley, we have these columns:
username : name : location : description : followers_count : following_count

We have 1 data rows for damianmarley in the follower data file.
For damianmarley we have 1 unique locations.
For damianmarley we have 0 words in the descriptions.
Here are the five most common words:
[]

----------------------------------------

We see 2 in your follower file for DermotKennedy, assuming a header row.
In the follower data file (DermotKennedy_followers_data.txt) for DermotKennedy, we have these columns:
username : name : location : description : followers_count : following_count

We have 2 data rows for DermotKennedy in the follower data file.
For DermotKennedy we have 1 unique locations.
For DermotKennedy we have 0 words in the descriptions.
Here are the five most common words:
[]

----------------------------------------



## For the section above, something is not correct because in the text file I have way more than 1 followers.

### Checking Lyrics
The output from your lyrics scrape should be stored in files located in this path from the directory: /lyrics/[Artist Name]/[filename from URL]. This code summarizes the information at a high level to help the instructor evaluate your work

In [129]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend((infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")

For damianmarley we have 41 files.
For damianmarley we have roughly 106712 words, 80 are unique.
For DermotKennedy we have 15 files.
For DermotKennedy we have roughly 27044 words, 68 are unique.


In [145]:
conda install nbconvert

Solving environment: ...working... done

# All requested packages already installed.

Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.
